In [ ]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
import statistics
import random
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss
import scipy.misc
# from scipy.misc import imread, imresize


In [ ]:
import pickle
import datetime
import pandas as pd
import keras

from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, \
                                       ZeroPadding2D

# from keras.layers.normalization import BatchNormalization
# from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import model_from_json
# from sklearn.metrics import log_loss
from numpy.random import permutation
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/deep_learning_project/

/content/drive/.shortcut-targets-by-id/15gKjVs5Brz2wVAZdIJoqScr7ejFRXBo9/deep_learning_project


# ResNet50

In [ ]:
#Load data
X_train=np.load('/content/drive/MyDrive/deep_learning_project/X_train.npy')
X_test=np.load('/content/drive/MyDrive/deep_learning_project/X_test.npy')
y_train=np.load('/content/drive/MyDrive/deep_learning_project/y_train.npy')
y_test=np.load('/content/drive/MyDrive/deep_learning_project/y_test.npy')

In [ ]:
#Load pretrained resnet50
import tensorflow.keras as K
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf

model_input_shape = (224,224,3)
base_model  =K.applications.resnet.ResNet50(include_top = False,
                                                  weights = 'imagenet',
                                                  input_shape = model_input_shape)
base_model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

In [ ]:
#Set Model
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

x = base_model.output
x = K.layers.Flatten()(x)
x = K.layers.Dropout(0.5)(x)

output =K.layers.Dense(units = 10,activation = tf.nn.softmax)(x)
model = K.models.Model(inputs=base_model.inputs, outputs=output)

model.compile(optimizer=K.optimizers.Adam(0.0001),
              loss=K.losses.CategoricalCrossentropy(from_logits = False),
              metrics=['accuracy'])

In [ ]:
#Run the model withou data agumentation
hist = model.fit(
    x = X_train,y=y_train,
    validation_data=(X_test,y_test),
    batch_size = 64,
    epochs=100,
    callbacks = [callback,time_callback],
    verbose=1)

Epoch 1/100
270/270 [==============================] - 58s 198ms/step - loss: 0.7466 - accuracy: 0.8483 - val_loss: 1.0245 - val_accuracy: 0.7701
Epoch 2/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0140 - accuracy: 0.9955 - val_loss: 0.8076 - val_accuracy: 0.8080
Epoch 3/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.7510 - val_accuracy: 0.8235
Epoch 4/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0121 - accuracy: 0.9967 - val_loss: 1.7037 - val_accuracy: 0.7748
Epoch 5/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0278 - accuracy: 0.9941 - val_loss: 0.7766 - val_accuracy: 0.8523
Epoch 6/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0172 - accuracy: 0.9967 - val_loss: 0.6524 - val_accuracy: 0.8570
Epoch 7/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0058 - accuracy: 0.9983 - val_loss: 0.9559 -

In [ ]:
#Save output
res_50_no_augment = pd.DataFrame(hist.history)
res_50_no_augment['time'] = time_callback.times
res_50_no_augment.to_csv('res_50_no_augment.csv')

In [ ]:
#Run model with data augmentation
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping
import time



#data augmentation
datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = 64)

# Fits the model on batches with real-time data augmentation:
hist = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, callbacks=[time_callback],
                                                   epochs = 50, verbose = 1, validation_data = (X_test, y_test))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
269/269 [==============================] - 215s 783ms/step - loss: 1.5817 - accuracy: 0.5884 - val_loss: 0.5572 - val_accuracy: 0.8664
Epoch 2/50
269/269 [==============================] - 210s 777ms/step - loss: 0.7090 - accuracy: 0.7684 - val_loss: 1.0661 - val_accuracy: 0.7563
Epoch 3/50
269/269 [==============================] - 215s 795ms/step - loss: 0.5608 - accuracy: 0.8183 - val_loss: 0.9638 - val_accuracy: 0.7707
Epoch 4/50
269/269 [==============================] - 215s 795ms/step - loss: 0.4683 - accuracy: 0.8413 - val_loss: 0.5267 - val_accuracy: 0.8808
Epoch 5/50
269/269 [==============================] - 214s 793ms/step - loss: 0.4129 - accuracy: 0.8596 - val_loss: 1.3406 - val_accuracy: 0.7739
Epoch 6/50
269/269 [==============================] - 213s 790ms/step - loss: 0.4200 - accuracy: 0.8618 - val_loss: 0.8258 - val_accuracy: 0.8602
Epoch 7/50
269/269 [==============================] - 214s 793ms/step - loss: 0.3485 - accuracy: 0.8825 - val_loss: 0.4853 -

In [ ]:
#Save output
res_50_with_augment = pd.DataFrame(hist.history)
res_50_with_augment['time'] = time_callback.times
res_50_with_augment.to_csv('res_50_with_augment.csv')